In [2]:
import datetime as dt
import itertools
from pathlib import Path

import cartopy.crs as ccrs
import dask
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams["animation.html"] = "jshtml"
import matplotlib.animation
import numpy as np
import pandas as pd
from scipy import stats
from scipy import interpolate
import xarray as xr

from mcs_prime import McsTracks, McsTrack, PATHS


Load all tracks from 2019/01-2020/01

In [6]:
tracks = McsTracks.open(PATHS['statsdir'] / 'mcs_tracks_final_extc_20190101.0000_20200101.0000.nc', 
                                 PATHS['pixeldir'])

In [7]:
tracks.dstracks

<xarray.Dataset>
Dimensions:                  (tracks: 32843, times: 400, mergers: 50, nmaxpf: 3)
Coordinates:
  * tracks                   (tracks) int64 0 1 2 3 ... 32839 32840 32841 32842
  * times                    (times) int64 0 1 2 3 4 5 ... 395 396 397 398 399
  * mergers                  (mergers) int64 0 1 2 3 4 5 6 ... 44 45 46 47 48 49
  * nmaxpf                   (nmaxpf) int64 0 1 2
Data variables: (12/55)
    track_duration           (tracks) int32 ...
    start_status             (tracks) float64 ...
    end_status               (tracks) float64 ...
    start_basetime           (tracks) datetime64[ns] 2019-01-01T00:30:00 ... ...
    end_basetime             (tracks) datetime64[ns] 2019-01-02T12:30:00 ... ...
    start_split_cloudnumber  (tracks) float64 ...
    ...                       ...
    pf_mcsstatus             (tracks, times) float64 ...
    movement_distance        (tracks, times) float32 ...
    movement_speed           (tracks, times) float32 ...
    movement_theta           (tracks, times) float32 ...
    movement_distance_x      (tracks, times) float32 ...
    movement_distance_y      (tracks, times) float32 ...
Attributes: (12/23)
    Title:                       Statistics of each MCS track
    Institution:                 Pacific Northwest National Laboratory
    Contact:                     Zhe Feng, zhe.feng@pnnl.gov
    Created_on:                  Mon Feb 28 05:50:54 2022
    startdate:                   20190101.0000
    enddate:                     20200101.0000
    ...                          ...
    PF_PF_min_majoraxis_thresh:  20
    coefs_pf_area:               [2874.05    89.825]
    coefs_pf_rr:                 [3.01657   0.0144461]
    coefs_pf_skew:               [0.194462  0.0100072]
    coefs_pf_heavyratio:         [3.419024 0.438709]
    max_speed_thresh:            50

In [17]:
(np.nansum(tracks.dstracks.mcs_status.values, axis=1) / tracks.dstracks.track_duration.values).mean()

0.6937189533224474

In [20]:
np.where(tracks.dstracks.mcs_status.values)

(array([    0,     0,     0, ..., 32842, 32842, 32842]),
 array([  0,   1,   2, ..., 397, 398, 399]))

In [21]:
mcs_status_change = np.zeros_like(tracks.dstracks.mcs_status.values)

In [22]:
mcs_status_change[:, 0] = tracks.dstracks.mcs_status.values[:, 0]

In [23]:
mcs_status_change[:, 1:] = tracks.dstracks.mcs_status.values[:, 1:] - tracks.dstracks.mcs_status.values[:, :-1]

In [48]:
mcs_status_change[np.sum(mcs_status_change == 1, axis=1) >= 3, :].shape

(1025, 400)

In [36]:
mcs_status_change.shape

(32843, 400)

In [50]:
a = np.array([0, 0, 1, 0, 0, 0, 1])

In [51]:
a

array([0, 0, 1, 0, 0, 0, 1])

In [75]:
mcs_status_first = np.zeros_like(tracks.dstracks.mcs_status.values, dtype=bool)
for i in range(len(tracks.dstracks.tracks)):
    for j in range(400):
        if tracks.dstracks.mcs_status.values[i, j]:
            mcs_status_first[i, j] = True
            break

In [78]:
(mcs_status_first.sum(axis=1) == 1).all()

True

In [67]:
np.nansum(tracks.dstracks.mcs_status.values, axis=1).min()

4.0

In [84]:
(tracks.dstracks.area.values < 4000).sum()

24879

In [83]:
tracks.dstracks.mcs_status

<xarray.DataArray 'mcs_status' (tracks: 32843, times: 400)>
array([[ 1.,  1.,  1., ..., nan, nan, nan],
       [ 1.,  1.,  1., ..., nan, nan, nan],
       [ 1.,  1.,  1., ..., nan, nan, nan],
       ...,
       [ 0.,  0.,  0., ..., nan, nan, nan],
       [ 0.,  0.,  1., ..., nan, nan, nan],
       [ 1.,  1.,  1., ..., nan, nan, nan]], dtype=float32)
Coordinates:
  * tracks   (tracks) int64 0 1 2 3 4 5 ... 32837 32838 32839 32840 32841 32842
  * times    (times) int64 0 1 2 3 4 5 6 7 8 ... 392 393 394 395 396 397 398 399
Attributes:
    long_name:  Flag indicating the status of MCS based on Tb. 1 = Yes, 0 = No
    units:      unitless

In [87]:
tracks.dstracks.ccs_area.values == tracks.dstracks.area.values

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [ True, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])